In [1]:
import pandas as pd

In [2]:
#df = pd.read_csv('/mnt/c/Users/Dleg/Data/MOPC/MOPC/Enriquecimiento de Datos/Data Cleaning/PyP/Datasets/Datasets enviados/Cartera_sin_enriquecer - Estados_idcliente - 2602-4.csv')
df = pd.read_csv('/mnt/c/Users/Dleg/Data/MOPC/MOPC/Enriquecimiento de Datos/Data Cleaning/Insiders/Datasets/Datasets enviados/Envio Insiders - Estado-IDCliente - 1603.csv')

In [3]:
df.head()

,IDMOROSO,NOMBRE,IDCLIENTE,ESTADO,FILTROA,DATOADIC
0,DNI 14384837,ACOSTA MARIA CRISTINA,FRANCES_2012,NO VISTO,3 - Ag 3ra,NaN
1,DNI 33803357,CARRASCO SERGIO DAMIAN,FRANCES_2012,NO VISTO,1 - Ag 1ra,NaN
2,DNI 34570749,TOLDI PRESENTADO SOLANGE GISELLE,FRANCES_2012,NO VISTO,1 - Ag 1ra,NaN
3,DNI 36991995,ANTECAO MAURO LEANDRO,FRANCES_2012,NO VISTO,2 - Ag 2ra,NaN
4,DNI 40359046,VATTOVAZ VICTORIA,FRANCES_2012,NO VISTO,1 - Ag 1ra,NaN


In [4]:
df.shape

(619, 6)

In [5]:
df['IDMOROSO'] = df['IDMOROSO'].str.strip()

In [6]:
df.loc[(df['IDCLIENTE'].str.startswith('AMEX')) , 'NRO_DOC'] = df['DATOADIC']
df.loc[~(df['IDCLIENTE'].str.startswith('AMEX')) , 'NRO_DOC'] = df['IDMOROSO']

In [7]:
df = df.dropna(subset=['NRO_DOC'])

In [8]:
df.shape

(619, 7)

In [9]:
df.head()

,IDMOROSO,NOMBRE,IDCLIENTE,ESTADO,FILTROA,DATOADIC,NRO_DOC
0,DNI 14384837,ACOSTA MARIA CRISTINA,FRANCES_2012,NO VISTO,3 - Ag 3ra,NaN,DNI 14384837
1,DNI 33803357,CARRASCO SERGIO DAMIAN,FRANCES_2012,NO VISTO,1 - Ag 1ra,NaN,DNI 33803357
2,DNI 34570749,TOLDI PRESENTADO SOLANGE GISELLE,FRANCES_2012,NO VISTO,1 - Ag 1ra,NaN,DNI 34570749
3,DNI 36991995,ANTECAO MAURO LEANDRO,FRANCES_2012,NO VISTO,2 - Ag 2ra,NaN,DNI 36991995
4,DNI 40359046,VATTOVAZ VICTORIA,FRANCES_2012,NO VISTO,1 - Ag 1ra,NaN,DNI 40359046


In [10]:
df.tail()

,IDMOROSO,NOMBRE,IDCLIENTE,ESTADO,FILTROA,DATOADIC,NRO_DOC
614,RMS376417243132000,CLAUDIA L ONUFRYJCZUK,AMEX_PARM,NO UBICADO,EDICOLA,27 32604085 7,27 32604085 7
615,RMS379977155103007,EDUARDO JUAN VISCONTI,AMEX_TNLM,NO UBICADO,MBESOSTRI,20041735504,20041735504
616,RMS376414028951001,FERNANDO PABLO KUKULJAN,AMEX_PARM,SKIP LEVEL1 OK,EDICOLA,20231294172,20231294172
617,RMS376456717431001,LUIS ALFREDO FRIAS,AMEX_TNCM,SKIP LEVEL1 OK,MBESOSTRI,30711477876,30711477876
618,RMS379977689141002,DANIEL ANDRES VELAZCO S,AMEX_ALLR,SKIP LEVEL1 OK,IGODOY,20257116744,20257116744


## 12.2) DNI Cleaning

Limpiamos el dataframe __df__, ya que dentro de DATO_A_ENVIAR tenemos 9 tipos de casos diferentes de datos:

- DNI 12345678
- DNI12345678
- DNIE1234678
- LC  1234567
- LE  1234567
- CUIT00123456780
- 00123456780 --> Es un CUIT.
- 00 12345678 0 --> Es un CUIT.
- DNI 00123456780 --> Dice DNI pero es un CUIT de una persona __jurídica__ (TO DO)

Para eso definimos 5 funciones:

- __Tipo_DOC__: Se definen dos funciones para determinar el tipo de DNI. Una inicial, y otra para corregir sobre el dataframe resultante.
- __Standard DNI__: Estandariza los DNI para 7 de los casos. Esta función debe ejecutarse en primer lugar.
- __DNI Spaces__: Elimina los espacios.
- __DNI Zero__: Elimina los ceros iniciales de ciertos CUITs para que coincidan con los DNI.
- __Cuit__: Decisión sobre dejar un cuit o volverlo DNI

In [11]:
def tipo_doc_1 (string_dni):
    
    if 'DNI' in string_dni:
        tipo = 'DNI'
    
    elif 'LC' in string_dni:
        tipo = 'LC'
    
    elif 'LE' in string_dni:
        tipo = 'LE'
    
    elif 'DNIE' in string_dni:
        tipo = 'DNIE'    
    
    elif ((string_dni.startswith(('CUIT30','CUIT33','CUIT34'))) & (len(string_dni)>8)):
        tipo = 'CUIT'
        
    elif ((string_dni.startswith(('30','33','34'))) & (len(string_dni)>8)):
        tipo = 'CUIT'

    elif ((string_dni.startswith(('20','23','24','27'))) & (len(string_dni)>8)):
        tipo = 'DNI'

    elif ((string_dni.startswith(('CUIT20','CUIT23','CUIT24','CUIT27'))) & (len(string_dni)>8)):
        tipo = 'DNI'
        
    else:
        tipo = 'DNI'
    
    return tipo

In [12]:
def tipo_doc_2 (string_dni, tipo_original):
    
    if (len(string_dni) > 8) & (tipo_original == 'DNI'):
        tipo = 'CUIT'
    else:
        tipo = tipo_original
    return tipo

In [13]:
def standard_dni (string_dni):
    """DNI separation by each case.
    """
    
    if 'DNI ' in string_dni:
        dni = string_dni.split('DNI ')[1]
        
    elif 'DNIE' in string_dni:
        dni = string_dni.split('DNIE')[1]
    
    elif 'DNI' in string_dni:
        dni = string_dni.split('DNI')[1]
    
    elif 'LC  ' in string_dni:
        dni = string_dni.split('LC  ')[1]
    
    elif 'LE  ' in string_dni:
        dni = string_dni.split('LE  ')[1]
    
    elif 'CUIT' in string_dni:
        dni = string_dni.split('CUIT')[1]
        
    else:
        dni = string_dni
        
    return dni

In [14]:
def dni_characters (string_dni):
    
    if ' ' in string_dni:
        dni = string_dni.replace(' ','')
        
    elif '-' in string_dni:
        dni = string_dni.replace('-','')

    else:
        dni = string_dni
    
    return dni

In [15]:
def cuit (string_dni):
    
    if ((string_dni.startswith('20')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]
        
    elif ((string_dni.startswith('23')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]
        
    elif ((string_dni.startswith('24')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]

    elif ((string_dni.startswith('27')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]

    else:
        cuit = string_dni
        
    return cuit

In [16]:
def dni_zero (string_dni):
    
    if string_dni.startswith('0'):
        dni = string_dni [1:]

    else:
        dni = string_dni
    
    return dni

En primer lugar, trimmeamos los espacios que sobran:

In [17]:
df['NRO_DOC'] = [(str(df['NRO_DOC'][i]).strip()) for i in range (df['NRO_DOC'].count())]

Se implementan estas funciones:

Para determinar el __TIPO DE DOCUMENTO__ en primer lugar:

In [18]:
df['TIPO_DOC'] = [tipo_doc_1(df['NRO_DOC'][i]) for i in range (df['NRO_DOC'].count())]

Para la limpieza de DNIs:

In [19]:
df['NRO_DOC'] = [standard_dni(df['NRO_DOC'][i]) for i in range (df['NRO_DOC'].count())]

In [20]:
df['NRO_DOC'] = [dni_characters(df['NRO_DOC'][i]) for i in range (df['NRO_DOC'].count())]

In [21]:
df['NRO_DOC'] = [cuit(df['NRO_DOC'][i]) for i in range (df['NRO_DOC'].count())]

In [22]:
df['NRO_DOC'] = [dni_zero(df['NRO_DOC'][i]) for i in range (df['NRO_DOC'].count())]

Para corregir el __TIPO DE DOCUMENTO__ sobre el dataframe resultante

In [23]:
df['TIPO_DOC'] = [tipo_doc_2(df['NRO_DOC'][i],df['TIPO_DOC'][i]) for i in range (df['NRO_DOC'].count())]

In [24]:
df.head()

,IDMOROSO,NOMBRE,IDCLIENTE,ESTADO,FILTROA,DATOADIC,NRO_DOC,TIPO_DOC
0,DNI 14384837,ACOSTA MARIA CRISTINA,FRANCES_2012,NO VISTO,3 - Ag 3ra,NaN,14384837,DNI
1,DNI 33803357,CARRASCO SERGIO DAMIAN,FRANCES_2012,NO VISTO,1 - Ag 1ra,NaN,33803357,DNI
2,DNI 34570749,TOLDI PRESENTADO SOLANGE GISELLE,FRANCES_2012,NO VISTO,1 - Ag 1ra,NaN,34570749,DNI
3,DNI 36991995,ANTECAO MAURO LEANDRO,FRANCES_2012,NO VISTO,2 - Ag 2ra,NaN,36991995,DNI
4,DNI 40359046,VATTOVAZ VICTORIA,FRANCES_2012,NO VISTO,1 - Ag 1ra,NaN,40359046,DNI


In [25]:
df.tail()

,IDMOROSO,NOMBRE,IDCLIENTE,ESTADO,FILTROA,DATOADIC,NRO_DOC,TIPO_DOC
614,RMS376417243132000,CLAUDIA L ONUFRYJCZUK,AMEX_PARM,NO UBICADO,EDICOLA,27 32604085 7,32604085,DNI
615,RMS379977155103007,EDUARDO JUAN VISCONTI,AMEX_TNLM,NO UBICADO,MBESOSTRI,20041735504,4173550,DNI
616,RMS376414028951001,FERNANDO PABLO KUKULJAN,AMEX_PARM,SKIP LEVEL1 OK,EDICOLA,20231294172,23129417,DNI
617,RMS376456717431001,LUIS ALFREDO FRIAS,AMEX_TNCM,SKIP LEVEL1 OK,MBESOSTRI,30711477876,30711477876,CUIT
618,RMS379977689141002,DANIEL ANDRES VELAZCO S,AMEX_ALLR,SKIP LEVEL1 OK,IGODOY,20257116744,25711674,DNI


In [26]:
df_export = df.rename(columns={'NRO_DOC':'DOCUMENTO','TIPO_DOC':'TIPO'})

In [27]:
df_export = df_export [['TIPO','DOCUMENTO','NOMBRE']]

In [28]:
df_export.head()

,TIPO,DOCUMENTO,NOMBRE
0,DNI,14384837,ACOSTA MARIA CRISTINA
1,DNI,33803357,CARRASCO SERGIO DAMIAN
2,DNI,34570749,TOLDI PRESENTADO SOLANGE GISELLE
3,DNI,36991995,ANTECAO MAURO LEANDRO
4,DNI,40359046,VATTOVAZ VICTORIA


In [29]:
df_export.shape

(619, 3)

In [30]:
df_export.to_csv ('/mnt/c/Users/Dleg/Data/MOPC/MOPC/Enriquecimiento de Datos/Data Cleaning/Insiders/Datasets/Datasets enviados/MOPC Envio 1603.csv',index=False)